##  Solar Gravitational Potential Energy

On one hand, remember that the gravitational potential energy is given by (equation (17))
\begin{equation*}
    \Omega_\odot = - \int_{0}^{R_\odot} \dfrac{GM(r)}{r}\ \text{d}M(r),
\end{equation*}
whith (equation (1) and (2), respectivelly)
\begin{align*}
    \text{d}M(r) &= \left( 4\pi r^2\text{d}r \right) \rho(r), \\
    M(r) &= 4\pi \int_0^r \rho(r')\ r'^2 \text{d}r'.
\end{align*}

So, we can substitute $\text{d}M(r)$ into $\Omega_\odot$, i.e.,
\begin{align*}
    \Omega_\odot &= - \int_{0}^{R_\odot} \dfrac{GM(r)}{r} \left(4\pi r^2 \text{d}r\right) \rho(r), \\
    \Omega_\odot &= - 4\pi G \int_{0}^{R_\odot} M(r)\rho(r)\ r \text{d}r,
\end{align*}
and now we can integrate numerically $\Omega_\odot$, with a previus cumulative numerical integration of $M(r)$. Let's to codify this!

First, let's read the data of $r$ and $\rho(r)$ from the file `cptrho.l5bi.d.15c`(downloaded from https://users-phys.au.dk/jcd/solar_models/),

In [4]:
import numpy as np
import pandas as pd

def readFile(fname,fnames):
    data = np.loadtxt(fname)    
    data = pd.DataFrame(data, columns = fnames)
    return data

# Set attributes
fname = 'cptrho.l5bi.d.15c'
fnames = ['r/R', 'c (cm/s)', 'rho (g/cm^3)',
          'p (dyn/cm^2)', 'Gamma_1', 'T (K)']

# Read the file.txt
df = readFile(fname, fnames)
display(df)

,r/R,c (cm/s),rho (g/cm^3),p (dyn/cm^2),Gamma_1,T (K)
0,1.000713,686438.80,3.292483e-09,9.455764e+02,1.640705,4.348196e+03
1,1.000705,686573.05,3.469053e-09,9.962773e+02,1.641361,4.348491e+03
2,1.000697,686706.53,3.656545e-09,1.050125e+03,1.641997,4.348819e+03
3,1.000689,686840.63,3.855586e-09,1.107302e+03,1.642618,4.349183e+03
4,1.000681,686977.49,4.066838e-09,1.168000e+03,1.643233,4.349587e+03
...,...,...,...,...,...,...
2477,0.001448,50466295.00,1.538629e+02,2.348911e+17,1.668285,1.566774e+07
2478,0.001429,50466263.00,1.538637e+02,2.348920e+17,1.668284,1.566776e+07
2479,0.001410,50466228.00,1.538645e+02,2.348929e+17,1.668285,1.566778e+07
2480,0.001391,50466228.00,1.538650e+02,2.348937e+17,1.668284,1.566779e+07


Ordering the data by the radius (this is important for the numerical integration!)

In [5]:
df = df.sort_values(by='r/R').reset_index(drop=True)
df

,r/R,c (cm/s),rho (g/cm^3),p (dyn/cm^2),Gamma_1,T (K)
0,0.000000,50465569.00,1.538894e+02,2.349248e+17,1.668285,1.566847e+07
1,0.001391,50466228.00,1.538650e+02,2.348937e+17,1.668284,1.566779e+07
2,0.001410,50466228.00,1.538645e+02,2.348929e+17,1.668285,1.566778e+07
3,0.001429,50466263.00,1.538637e+02,2.348920e+17,1.668284,1.566776e+07
4,0.001448,50466295.00,1.538629e+02,2.348911e+17,1.668285,1.566774e+07
...,...,...,...,...,...,...
2477,1.000681,686977.49,4.066838e-09,1.168000e+03,1.643233,4.349587e+03
2478,1.000689,686840.63,3.855586e-09,1.107302e+03,1.642618,4.349183e+03
2479,1.000697,686706.53,3.656545e-09,1.050125e+03,1.641997,4.348819e+03
2480,1.000705,686573.05,3.469053e-09,9.962773e+02,1.641361,4.348491e+03


Now, we can integrate $M(r)$ as

In [7]:
import scipy.integrate as spi
from astropy import constants as const
from astropy import units as u

def computeMass(r, rho):
    y = rho*r**2
    integral = spi.cumtrapz(y, r, initial=0)  # cumulative numerical integration
    M = 4*np.pi*integral
    return M

# Adjust radius unit
r = df[fnames[0]]           # radius in units of R_sun
Rsun = const.R_sun.to(u.cm) # solar radius in cm
df['r (cm)'] = r*Rsun       # radius in cm

df['M (g)'] = computeMass(r = df['r (cm)'], rho = df['rho (g/cm^3)'] )
df

,r/R,c (cm/s),rho (g/cm^3),p (dyn/cm^2),Gamma_1,T (K),r (cm),M (g)
0,0.000000,50465569.00,1.538894e+02,2.349248e+17,1.668285,1.566847e+07,0.000000e+00,0.000000e+00
1,0.001391,50466228.00,1.538650e+02,2.348937e+17,1.668284,1.566779e+07,9.677883e+07,8.763161e+26
2,0.001410,50466228.00,1.538645e+02,2.348929e+17,1.668285,1.566778e+07,9.807979e+07,9.001948e+26
3,0.001429,50466263.00,1.538637e+02,2.348920e+17,1.668284,1.566776e+07,9.940162e+07,9.251141e+26
4,0.001448,50466295.00,1.538629e+02,2.348911e+17,1.668285,1.566774e+07,1.007374e+08,9.509778e+26
...,...,...,...,...,...,...,...,...
2477,1.000681,686977.49,4.066838e-09,1.168000e+03,1.643233,4.349587e+03,6.961736e+10,1.986526e+33
2478,1.000689,686840.63,3.855586e-09,1.107302e+03,1.642618,4.349183e+03,6.961792e+10,1.986526e+33
2479,1.000697,686706.53,3.656545e-09,1.050125e+03,1.641997,4.348819e+03,6.961848e+10,1.986526e+33
2480,1.000705,686573.05,3.469053e-09,9.962773e+02,1.641361,4.348491e+03,6.961903e+10,1.986526e+33


In [8]:
Deltax = df['r (cm)'][2480] - df['r (cm)'][2481]
Deltax

96778827.0

Remember that the solar mass is aproximately $M_\odot = 1.989 \times 10^{30}\ \text{kg} = 1.989 \times 10^{33}\ \text{g}$. So, from the row 2481 we can see our sucess! Let's see the difference between the computed value and the expected value (stored in the package `scipy.constants`)

In [4]:
Msun = df['M (g)'].max() * u.g
print('Solar mass computed by us:\n ', Msun)

MsunExpected = const.M_sun.to(u.g)
print('Solar mass expected (stored in scipy.constants):\n ', MsunExpected)

Error = np.abs(Msun - MsunExpected)/MsunExpected
Error = "{:.2%}".format(Error)
print('Relative difference:\n ', Error)

Solar mass computed by us:
  1.986525824523492e+33 g
Solar mass expected (stored in scipy.constants):
  1.988409870698051e+33 g
Relative difference:
  0.09%


Finally, let's compute $\Omega_\odot$

In [13]:
def computeGPE(r, rho, M):
    G = const.G                               # gravitational constant in m^3/kgs^2
    y = M*rho*r
    integral = spi.cumtrapz(y, r, initial=0)  # numerical integration
    integral = integral * (u.g**2 / u.cm)     # integral in units g^2/cm
        
    Omega = (-4*np.pi*G*integral ).to(u.erg)  # energy in erg
    return Omega

# reload the variables
Omega = computeGPE(r = df['r (cm)'], rho = df['rho (g/cm^3)'], M = df['M (g)'])
df['Omega (erg)'] = Omega.value
df

,r/R,c (cm/s),rho (g/cm^3),p (dyn/cm^2),Gamma_1,T (K),r (cm),M (g),Omega (erg)
0,0.000000,50465569.00,1.538894e+02,2.349248e+17,1.668285,1.566847e+07,0.000000e+00,0.000000e+00,-0.000000e+00
1,0.001391,50466228.00,1.538650e+02,2.348937e+17,1.668284,1.566779e+07,9.677883e+07,8.763161e+26,-5.295988e+38
2,0.001410,50466228.00,1.538645e+02,2.348929e+17,1.668285,1.566778e+07,9.807979e+07,9.001948e+26,-5.441294e+38
3,0.001429,50466263.00,1.538637e+02,2.348920e+17,1.668284,1.566776e+07,9.940162e+07,9.251141e+26,-5.595028e+38
4,0.001448,50466295.00,1.538629e+02,2.348911e+17,1.668285,1.566774e+07,1.007374e+08,9.509778e+26,-5.756851e+38
...,...,...,...,...,...,...,...,...,...
2477,1.000681,686977.49,4.066838e-09,1.168000e+03,1.643233,4.349587e+03,6.961736e+10,1.986526e+33,-6.110234e+48
2478,1.000689,686840.63,3.855586e-09,1.107302e+03,1.642618,4.349183e+03,6.961792e+10,1.986526e+33,-6.110234e+48
2479,1.000697,686706.53,3.656545e-09,1.050125e+03,1.641997,4.348819e+03,6.961848e+10,1.986526e+33,-6.110234e+48
2480,1.000705,686573.05,3.469053e-09,9.962773e+02,1.641361,4.348491e+03,6.961903e+10,1.986526e+33,-6.110234e+48


Therefore, the solar GPE computed by numerical integration, $\Omega_\odot$, is

In [6]:
OmegaSun = df['Omega (erg)'][2481] * u.erg
OmegaSun

<Quantity -6.11023353e+48 erg>

On the other hand, computing $\Omega_\odot$ as
\begin{equation*}
    \Omega_\odot = -\dfrac{GM_\odot^2}{R_\odot},
\end{equation*}
leads to

In [7]:
G = const.G        # gravitational constant G in m^3/kgs^2
Msun = const.M_sun # solar mass in kg
Rsun = const.R_sun # solar radius in m

OmegaSunExpected = (-G*Msun**2/Rsun).to(u.erg)
OmegaSunExpected

<Quantity -3.7931109e+48 erg>

Summarizing, we can see the relative difference

In [13]:
print('Solar GPE (Gravitational Potential Energy) computed by us:\n ', OmegaSun)

print('Solar GEP expected (constanst given by scipy.constants):\n ', OmegaSunExpected)

Error = np.abs((OmegaSun - OmegaSunExpected)/OmegaSunExpected)
Error = "{:.2%}".format(Error)
print('Relative difference:\n ', Error)
print('q =', OmegaSun/OmegaSunExpected)

Solar GPE (Gravitational Potential Energy) computed by us:
  -6.110233530610977e+48 erg
Solar GEP expected (constanst given by scipy.constants):
  -3.79311090499386e+48 erg
Relative difference:
  61.09%
q = 1.6108765822181694


This high relative difference (greater than 5%), could comes from the fact that we are computing $\Omega_\odot$ under different models, i.e., the fist model is the *Christensen-Dalsgaard et al* model and second model is the equation
\begin{equation*}
    \Omega_\odot = -\dfrac{GM_\odot^2}{R_\odot}.
\end{equation*}

A second aprouch could be trying a different numerical integration method for $\Omega_\odot$. Let's see if we obtain a different result

In [9]:
def computeGPE2(r, rho, M):
    G = const.G                               # gravitational constant in m^3/kgs^2    
    y = M*rho*r
    
    integral = [spi.trapz(y, r),
                spi.simps(y, r)]
    integral = integral * (u.g**2 / u.cm)     # integral in units g^2/cm
    
    
    Omega = ( -4*np.pi*G*integral ).to(u.erg) # energy in erg
    return Omega

# reload the variables
computeGPE2(r = df['r (cm)'], rho = df['rho (g/cm^3)'], M = df['M (g)'])

<Quantity [-6.11023353e+48, -6.11023921e+48] erg>

Ok, the results are approximately equals, therefore, we can conclude that the relative difference of 61% comes from we are computing $\Omega_\odot$ under differents models.